
# Testing Custom Mask R-CNN Model

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from keras.preprocessing.image import load_img, array_to_img, img_to_array

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

#import custom

# Root directory of the project
ROOT_DIR = "D:\Youtube_MaskRCNN"

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


# For model testing, uncomment this and load your model.



In [ ]:
WEIGHTS_PATH = "D:\\Youtube_MaskRCNN\\mask_rcnn_object_0100.h5"   # change it

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 4  # Background + rice pudding, plain rice, lentils
    
    BATCH_SIZE = 4

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 78

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
config = CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR, "/dataset/")
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.8

config = InferenceConfig()
config.display()

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "egg")
        self.add_class("object", 2, "rice")
        self.add_class("object", 3, "lentils")
        self.add_class("object", 4, "spinach")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        annotations1 = json.load(open(os.path.join(dataset_dir, 'via_region_data.json')))
        annotations = list(annotations1.values())
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            image_id = "{}_{}".format(subset, a['filename'])
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['name'] for s in a['regions']]
            name_dict = {"egg": 1, "rice": 2, "lentils": 3, "spinach":4}
            num_ids = [name_dict[a] for a in objects]

            self.add_image(
                "object",
                image_id=image_id,
                path=image_path,
                width=width,
                height=height,
                polygons=polygons,
                num_ids=num_ids
            )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
# Inspect the model in training or inference modes values: 'inference' or 'training'
TEST_MODE = "inference"
ROOT_DIR = "D:\Youtube_MaskRCNN\dataset"

def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

# Load validation dataset
# Must call before using the dataset
CUSTOM_DIR = "D:\Youtube_MaskRCNN\dataset"
dataset_val = CustomDataset()
dataset_val.load_custom(CUSTOM_DIR, "val")
dataset_val.prepare()
print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))

In [ ]:
dataset_train = CustomDataset()
dataset_train.load_custom(CUSTOM_DIR, "train")
dataset_train.prepare()

# To test the model, use the following codes:
# To test the metrics, skip all these codes:

In [ ]:
config = CustomConfig()
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir= MODEL_DIR, config=config)

In [ ]:
# Load COCO weights Or, load the last model you trained
weights_path = WEIGHTS_PATH
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
#Now, we are ready for testing our model on any image.
class_names = dataset_val.class_names

img = load_img("D:\\Youtube_MaskRCNN\\dataset\lentils_plainRice_spinach_35.jpeg")
img_arr = np.array(img)
results = model.detect([img_arr], verbose=1)
r = results[0]
visualize.display_instances(img_arr, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], figsize=(5,5))



In [ ]:
# testing images using test folder.

real_test_dir = "D:\Youtube_MaskRCNN\\friedEGG"
image_paths = []
for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))
class_names = dataset_val.class_names
count = 0
for image_path in image_paths:
    img = load_img(image_path)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose=1)
    r = results[0]
    count += 1
    visualize.display_instances(img_arr, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], figsize=(5,5))

# To test the metrics, directly run this!

In [ ]:
import os

config = CustomConfig()

# Define the directory where the models are stored
model_dir = "D:\\Youtube_MaskRCNN\\logs\\test"

#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=model_dir, config=config)

# Define a list to store the metrics for each epoch
epoch_metrics = []

# Iterate over each epoch from 1 to 50
for epoch in range(1, 100):
    # Load the model for the current epoch
    weights_path = os.path.join(model_dir, f"mask_rcnn_object_00{epoch}.h5")
    model.load_weights(weights_path, by_name=True)

    # Initialize lists to store metrics for the current epoch
    epoch_recall = []
    epoch_precision = []
    epoch_f1 = []
    epoch_mAP =[]
    
    image_ids = dataset_val.image_ids

    APs = []
    
    # Iterate over images
    for image_id in image_ids:
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_train, config,
                                   image_id, use_mini_mask=False)
        molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
        
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=0.5)
        
        APs.append(AP)
        
        # Append precision and recall for the current image
        epoch_recall.append(np.mean(recalls))
        epoch_precision.append(np.mean(precisions))
        epoch_mAP.append(np.mean(APs))
        
        # Compute F1 score
        f1_score = (2 * (np.mean(precisions) * np.mean(recalls))) / (np.mean(precisions) + np.mean(recalls))
        epoch_f1.append(f1_score)
    
    # Compute average precision, recall, and F1 score for the epoch
    avg_recall = np.mean(epoch_recall)
    avg_precision = np.mean(epoch_precision)
    avg_f1 = np.mean(epoch_f1)
    avg_mAP = np.mean(epoch_mAP)

    # Store the metrics for the current epoch
    epoch_metrics.append({"Epoch": epoch, "Precision": avg_precision, "Recall": avg_recall, "F1": avg_f1, "mAP": avg_mAP})

# Print the metrics for each epoch
for metrics in epoch_metrics:
    print(f"Epoch {metrics['Epoch']}: Precision - {metrics['Precision']}, Recall - {metrics['Recall']}, F1 - {metrics['F1']}, mAP - {metrics['mAP']}")


In [ ]:
import os
 
config = CustomConfig()
 
# Define the directory where the models are stored
model_dir = "D:\mask_rcnn\logs\object20240220T0020"
 
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=model_dir, config=config)
 
# Define a list to store the metrics for each epoch
epoch_metrics = []
 
# Iterate over each epoch from 1 to 100
for epoch in range(1, 101):
    # Load the model for the current epoch
    weights_path = os.path.join(model_dir, f"mask_rcnn_object_{epoch:04d}.h5")
    model.load_weights(weights_path, by_name=True)
 
    # Initialize lists to store metrics for the current epoch
    epoch_recall = []
    epoch_precision = []
    epoch_f1 = []
    epoch_mAP =[]
    
    image_ids = dataset_val.image_ids

    APs = []
    # Iterate over images
    for image_id in image_ids:
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_train, config,
                                   image_id, use_mini_mask=False)
        molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=0.5)
        APs.append(AP)
        # Append precision and recall for the current image
        epoch_recall.append(np.mean(recalls))
        epoch_precision.append(np.mean(precisions))
        epoch_mAP.append(np.mean(APs))
        # Compute F1 score
        f1_score = (2 * (np.mean(precisions) * np.mean(recalls))) / (np.mean(precisions) + np.mean(recalls))
        epoch_f1.append(f1_score)
    # Compute average precision, recall, and F1 score for the epoch
    avg_recall = np.mean(epoch_recall)
    avg_precision = np.mean(epoch_precision)
    avg_f1 = np.mean(epoch_f1)
    avg_mAP = np.mean(epoch_mAP)
 
    # Store the metrics for the current epoch
    epoch_metrics.append({"Epoch": epoch, "Precision": avg_precision, "Recall": avg_recall, "F1": avg_f1, "mAP": avg_mAP})
 
# Print the metrics for each epoch
for metrics in epoch_metrics:
    print(f"Epoch {metrics['Epoch']}: Precision - {metrics['Precision']}, Recall - {metrics['Recall']}, F1 - {metrics['F1']}, mAP - {metrics['mAP']}")

In [ ]:
# Extracting data for plotting
epochs = [metrics['Epoch'] for metrics in epoch_metrics]
precisions = [metrics['Precision'] for metrics in epoch_metrics]
recalls = [metrics['Recall'] for metrics in epoch_metrics]
f1_scores = [metrics['F1'] for metrics in epoch_metrics]
mAPs = [metrics['mAP'] for metrics in epoch_metrics]

# Plotting
plt.figure(figsize=(15, 5))


plt.subplot(1, 4, 1)
plt.plot(epochs, precisions, marker='o')
plt.title('Epochs vs Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')

plt.subplot(1, 4, 2)
plt.plot(epochs, recalls, marker='o')
plt.title('Epochs vs Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')

plt.subplot(1, 4, 3)
plt.plot(epochs, f1_scores, marker='o')
plt.title('Epochs vs F1 Score')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')

plt.subplot(1, 4, 4)
plt.plot(epochs, mAPs, marker='o')
plt.title('Epochs vs mAP')
plt.xlabel('Epochs')
plt.ylabel('mAP')

plt.tight_layout()
plt.show()

In [ ]:
# Compute VOC-Style mAP @ IoU=0.75
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_train, config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=0.75)
    APs.append(AP)
print("mAP at IoU threshold 0.75: ", np.mean(APs))
print(APs)

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
recall = []
precision = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_train, config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    _, precisions, recalls, _ =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=0.75)
    recall.append(np.mean(recalls))
    precision.append(np.mean(precisions))
print("F1: ", (2* (np.mean(precisions) * np.mean(recalls)))/(np.mean(precisions) + np.mean(recalls)))
print("recall: ", np.mean(recall))
print("Precision: ", np.mean(precision))

In [ ]:
# Define the path to the folder containing your data
folder_path = "D:\\Youtube_MaskRCNN\\dataset\\val"

# Initialize lists to store precision, recall, and AP values for all images
all_precisions = []
all_recalls = []
all_APs = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Assuming you load ground truth and predictions for each image, implement your loading function
    # gt_bbox, gt_class_id, gt_mask, pred_bbox, pred_class_id, pred_scores, pred_mask = load_data_from_file(os.path.join(folder_path, filename))
    
    # Compute precision, recall, and AP for the current image
    AP, precisions, recalls, _ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          pred_bbox, pred_class_id, pred_scores, pred_mask)
    
    # Append the values to the lists
    all_APs.append(AP)
    all_precisions.extend(precisions)
    all_recalls.extend(recalls)

# Compute the mean average precision (mAP) over all images
mAP = np.mean(all_APs)




In [ ]:
for class_id in range(1, dataset_val.num_classes):  # Assuming class IDs start from 1
        # Check if overlaps for the current class exist
        if class_id in overlaps:
            visualize.plot_overlaps(gt_class_id, r['class_ids'], r['scores'], overlaps[class_id],
                                    dataset_val.class_names[class_id])

In [ ]:
# Generate RPN trainig targets
# target_rpn_match is 1 for positive anchors, -1 for negative anchors
# and 0 for neutral anchors.
target_rpn_match, target_rpn_bbox = modellib.build_rpn_targets(
    image.shape, model.anchors, gt_class_id, gt_bbox, model.config)
log("target_rpn_match", target_rpn_match)
log("target_rpn_bbox", target_rpn_bbox)

positive_anchor_ix = np.where(target_rpn_match[:] == 1)[0]
negative_anchor_ix = np.where(target_rpn_match[:] == -1)[0]
neutral_anchor_ix = np.where(target_rpn_match[:] == 0)[0]
positive_anchors = model.anchors[positive_anchor_ix]
negative_anchors = model.anchors[negative_anchor_ix]
neutral_anchors = model.anchors[neutral_anchor_ix]
log("positive_anchors", positive_anchors)
log("negative_anchors", negative_anchors)
log("neutral anchors", neutral_anchors)

# Apply refinement deltas to positive anchors
refined_anchors = utils.apply_box_deltas(
    positive_anchors,
    target_rpn_bbox[:positive_anchors.shape[0]] * model.config.RPN_BBOX_STD_DEV)
log("refined_anchors", refined_anchors, )



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Function to get predictions for the validation dataset
def get_predictions(model, dataset):
    predictions = []

    for i in range(len(dataset.image_ids)):
        image, _, gt_class_id, gt_bbox = modellib.load_image_gt(dataset, config, i)
        molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
        
        # Use batch size of 1 for predictions
        results = model.detect([image], verbose=0)
        r = results[0]

        # Assuming that your model outputs class_ids, scores, and bounding boxes
        predictions.append({
            'class_ids': r['class_ids'],
            'scores': r['scores'],
            'rois': r['rois'],  # Bounding boxes
        })

    return predictions

# Get ground truth labels and model predictions
all_ground_truth = [modellib.load_image_gt(dataset_val, config, i) for i in range(len(dataset_val.image_ids))]

# Extract true labels
true_labels = [
    [
        dataset_val.class_names[label] if label != 0 else 'BG'  # Handle 'BG' class
        for label in gt_info[2]
    ] for gt_info in all_ground_truth
]

# Call get_predictions once for the entire dataset
all_predictions = get_predictions(model, dataset_val)

# Extract predicted labels
predicted_labels = [
    [
        dataset_val.class_names[label] if label != 0 else 'BG'  # Handle 'BG' class
        for label in p['class_ids'] if label != 0
    ] for p in all_predictions
]

# Extract detected class IDs for each instance
detected_class_ids = [
    [dataset_val.class_names[label] if label != 0 else 'BG' for label in p['class_ids']]
    for p in all_predictions
]

# Flatten the ground truth, predicted labels, and detected class IDs
flat_true_labels = [label for sublist in true_labels for label in sublist]
flat_predicted_labels = [label for sublist in predicted_labels for label in sublist]
flat_detected_class_ids = [label for sublist in detected_class_ids for label in sublist]

# Make sure lengths match by truncating the longer list
min_length = min(len(flat_true_labels), len(flat_predicted_labels), len(flat_detected_class_ids))
flat_true_labels = flat_true_labels[:min_length]
flat_predicted_labels = flat_predicted_labels[:min_length]
flat_detected_class_ids = flat_detected_class_ids[:min_length]

# Create confusion matrix
unique_labels = [label for label in dataset_val.class_names if label != 'BG']  # Exclude 'BG'
cm = confusion_matrix(flat_true_labels, flat_predicted_labels, labels=unique_labels)

# Visualize confusion matrix using seaborn
plt.figure(figsize=(len(unique_labels), len(unique_labels)))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Additional: Classification report
print("Classification Report:\n", classification_report(
    flat_true_labels, 
    flat_predicted_labels, 
    labels=[label for label in range(1, len(unique_labels) + 1)],  # Exclude 'BG'
    target_names=unique_labels
))


In [ ]:
# Grid of ground truth objects and their predictions
AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])

visualize.plot_overlaps(gt_class_id, r['class_ids'], r['scores'],
                        overlaps, dataset.class_names)

In [ ]:
# Display positive anchors before refinement (dotted) and
# after refinement (solid).
visualize.draw_boxes(image, boxes=positive_anchors, refined_boxes=refined_anchors, ax=get_ax())

In [ ]:
# Display a random sample of proposals.
# Proposals classified as background are dotted, and
# the rest show their class and confidence score.
limit = 200
ixs = np.random.randint(0, proposals.shape[0], limit)
captions = ["{} {:.3f}".format(dataset.class_names[c], s) if c > 0 else ""
            for c, s in zip(roi_class_ids[ixs], roi_scores[ixs])]
visualize.draw_boxes(image, boxes=proposals[ixs],
                     visibilities=np.where(roi_class_ids[ixs] > 0, 2, 1),
                     captions=captions, title="ROIs Before Refinement",
                     ax=get_ax())

In [ ]:
# Run RPN sub-graph
pillar = model.keras_model.get_layer("ROI").output  # node to start searching from

# TF 1.4 and 1.9 introduce new versions of NMS. Search for all names to support TF 1.3~1.10
nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression:0")
if nms_node is None:
    nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression/NonMaxSuppressionV2:0")
if nms_node is None: #TF 1.9-1.10
    nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression/NonMaxSuppressionV3:0")

rpn = model.run_graph([image], [
    ("rpn_class", model.keras_model.get_layer("rpn_class").output),
    ("pre_nms_anchors", model.ancestor(pillar, "ROI/pre_nms_anchors:0")),
    ("refined_anchors", model.ancestor(pillar, "ROI/refined_anchors:0")),
    ("refined_anchors_clipped", model.ancestor(pillar, "ROI/refined_anchors_clipped:0")),
    ("post_nms_anchor_ix", nms_node),
    ("proposals", model.keras_model.get_layer("ROI").output),
])

In [ ]:
# Show top anchors by score (before refinement)
limit = 100
sorted_anchor_ids = np.argsort(rpn['rpn_class'][:,:,1].flatten())[::-1]
visualize.draw_boxes(image, boxes=model.anchors[sorted_anchor_ids[:limit]], ax=get_ax())

In [ ]:
# Show top anchors with refinement. Then with clipping to image boundaries
limit = 50
ax = get_ax(1, 2)
pre_nms_anchors = utils.denorm_boxes(rpn["pre_nms_anchors"][0], image.shape[:2])
refined_anchors = utils.denorm_boxes(rpn["refined_anchors"][0], image.shape[:2])
refined_anchors_clipped = utils.denorm_boxes(rpn["refined_anchors_clipped"][0], image.shape[:2])
visualize.draw_boxes(image, boxes=pre_nms_anchors[:limit],
                     refined_boxes=refined_anchors[:limit], ax=ax[0])
visualize.draw_boxes(image, refined_boxes=refined_anchors_clipped[:limit], ax=ax[1])

In [ ]:
display_images(np.transpose(gt_mask, [2, 0, 1]), cmap="Blues")